# CloudEx Test Trading Session

### CloudEx Offline Session 5-7pm (PST)
* [0. Setup/Imports](#0.-Setup/Imports)
	* [Setup Libraries:](#Setup-Libraries:)
	* [Add Utility Functions](#Add-Utility-Functions)
* [1. Backtesting](#1.-Backtesting)
	* [1.1 Create Mean Reversion Trading Class](#1.1-Create-Mean-Reversion-Trading-Class)
	* [1.2 Get Symbol Historical Data](#1.2-Get-Symbol-Historical-Data)
	* [1.3 Backtest Mean Reversion Trader](#1.3-Backtest-Mean-Reversion-Trader)

### CloudEx Live Session 7-9pm (PST)
* [2. Connect to Live Exchange](#2.-Connect-to-Live-Exchange)
	* [2.1 Create Trader Object](#2.1-Create-Trader-Object)
	* [2.2 Get My Outstanding and Historical Orders](#2.2-Get-My-Outstanding-and-Historical-Orders)
	* [2.3 Get My Historical Trades](#2.3-Get-My-Historical-Trades)
	* [2.4 Get My Portfolio Matrix](#2.4-Get-My-Portfolio-Matrix)
* [3. Run Trading Algorithm in Live Environment](#3.-Run-Trading-Algorithm-in-Live-Environment)
	* [3.1 Create and Start Mean Reversion Trader](#3.1-Create-and-Start-Mean-Reversion-Trader)


# 0. Setup/Imports

For this section, execute the following cells. We will import the required packages and add some utility functions for handling CloudEx data structures.

## Setup Libraries:

In [1]:
# Import packages.
import datetime
import json
import os
import sys
import time

import pandas as pd
import numpy as np
import redis
from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# CloudEx imports.
import cloud_ex

# Import AlgorithmicTrader helper class.
from algorithmic_trader import AlgorithmicTrader
from algorithmic_trader import summarize_historical_trades_df

# Start Redis and its Python API.
os.system("redis-server --daemonize yes")
time.sleep(1)

# Get CloudEX and VM-specific config. 
# NOTE: gateway_ip will be null when the exchange is not online 
def get_vm_config():
    with open("vm_config.json", "r") as read_file:
        config = json.load(read_file)
    return config

config = get_vm_config()

In [2]:
config

{'gateway_ip': '10.128.15.220',
 'client_id': 'C15',
 'client_token': 'gaqxbfhabzvewlhdkowlimilhglzkhjx',
 'project_id': 'jinkun-pro1',
 'bigtable_id': 'test-bt',
 'table_name': 'record-tb'}

## Add Utility Functions

In [3]:
ORDER_FIELDS_LIST = [
    'Symbol', 'OrderID', 'CancelID', 'ClientID', 'OrderType', 'OrderAction',
    'SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
    'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice', 'ResultType','NumShares'
]

TRADE_FIELDS_LIST = [
    "Symbol", "BuyerSerialNum", "SellerSerialNum", "BuyerOrderID",
    "SellerOrderID", "BuyerClientID", "SellerClientID", "ExecPrice",
    "CashTraded", "SharesTraded", "CreationTimestamp", "ReleaseTimestamp",
    "TradeSerialNum"
]

'''
Takes in a cloud_ex.VectorOrder with serialized orders and returns a DataFrame
'''
def OrderDF(order_vec):
    if not len(order_vec):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = pd.DataFrame(order_vec).applymap(lambda x:x.SerializeOrder())[0].str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.VectorOrder with serialized trades and returns a DataFrame
'''
def TradeDF(trade_vec):
    if not len(trade_vec):
        return pd.DataFrame(columns=TRADE_FIELDS_LIST)
    df = pd.DataFrame(trade_vec).applymap(lambda x:x.SerializeTrade())[0].str.split('|', expand=True)
    df.columns = TRADE_FIELDS_LIST
    for label in ["ExecPrice", "CashTraded", "SharesTraded",
                  "CreationTimestamp", "ReleaseTimestamp", "TradeSerialNum"]:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.MapStringOrder mapping Order ID strings to outstanding the coorresponding orders, 
and returns a DataFrame
'''
def OutstandingOrderDF(outstanding_orders):
    if not len(outstanding_orders):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = (pd.DataFrame(outstanding_orders.items())[1]).apply(lambda x:x.SerializeOrder()).str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df 

# 1. Backtesting

Let's see how we can backtest our trading algorithms to get them ready for live trading. In the following cells we will download historical data and evaluate how well a mean reversion trader would have done.

## 1.1 Create Mean Reversion Trading Class

In [4]:
class meanReversionTrader(AlgorithmicTrader):
    def __init__(self, trader, symbol_list, bin_interval_ms=500):
        """
        Mean Reversion Trader Class.

        :param trader(cloud_ex.Trader): CloudEx's base Trader object.
        :param symbol_list: List of ticker symbols (str) to fetch data for.
        :param bin_interval_ms: Frequency with which to bin trading data in milliseconds (int).
        """
        # Initialize AlgorithmicTrader.
        AlgorithmicTrader.__init__(self, trader, symbol_list, bin_interval_ms=bin_interval_ms)

    def algorithm(self, df, **kwargs):
        """ 
        Calculate buy sell points for an equity.
        A buy is triggered when the moving average is threshold% below the price.
        A sell is triggered when the moving average is threshold% above the price.
        For this implementation we evaluate the stock price at the close of each bin_interval_ms time interval.

        :param df: Dataframes with prices for an equity (pd.DataFrame).
        Mean reversion specific arguments:
          :param ma: Number of bins to consider in the moving average (int).
          :param threshold: Amount ma is below/above price (as a percentage) for which to buy/sell (float).

        Returns: 
            An (action, price) pair.
            
            'action' is None when no action is determined to be taken. This happens either when we do not have enough
            data to build a moving average estimate or if the price is within a small threshold above/below
            the moving average. In this case, 'price' is also None.

            'action' is 'Buy' when the price rises above the threshold defined for the moving average. 
                The buy price is set to the most recent closing price.
                
            'action' is 'Sell' when the price drops below the threshold defined for the moving average. 
                The sell price is set to the most recent closing price.
        """
        # Unpack keyword arguments.
        ma = kwargs['ma']
        threshold = kwargs['threshold']
        
        # Handle case when data not available to build moving average.
        if len(df) < ma:
            return None, None
        
        # Select the last ma rows from our dataframe for processing.
        df = df.iloc[-1*ma:]

        df['MA'] = df['ClosePrice'].rolling(ma).mean()
        row = df.iloc[-1]

        # Check if moving average price is defined.
        # This value will be undefined for the first `ma` steps.
        if np.isnan(row['MA']):
            return None, None

        buy_cutoff = (1.0 - threshold / 100.0) * row['MA']
        sell_cutoff = (1.0 + threshold / 100.0) * row['MA']
        p_s = row['ClosePrice']
        if p_s <= buy_cutoff:
            return 'Buy', p_s
        elif p_s >= sell_cutoff:
            return 'Sell', p_s
        else:
            return None, None

## 1.2 Get Symbol Historical Data

In [5]:
symbol = 'AA'
seconds_in_past = 60*60*24 # One day
end_time_ms = int(time.time()*1e3)
start_time_ms = end_time_ms - int(seconds_in_past*1e3)

symbol_trades_vec = cloud_ex.VectorTrade()

cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
print("There are a total of {} trade(s) in the last day.".format(len(symbol_trades_vec)))
symbol_historical_trades_df = TradeDF(symbol_trades_vec)

symbol_historical_trades_df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
symbol_historical_trades_df

There are a total of 11499 trade(s) in the last day.


,Symbol,BuyerSerialNum,SellerSerialNum,BuyerOrderID,SellerOrderID,BuyerClientID,SellerClientID,ExecPrice,CashTraded,SharesTraded,CreationTimestamp,ReleaseTimestamp,TradeSerialNum
0,AA,0,0,NULL,NULL,NULL,NULL,6,6,1,1603331001572611,1603331001572961,34
1,AA,0,0,NULL,NULL,NULL,NULL,6,6,1,1603331001584590,1603331001584940,36
2,AA,0,0,NULL,NULL,NULL,NULL,6,6,1,1603331002964052,1603331002964402,129
3,AA,0,0,NULL,NULL,NULL,NULL,6,6,1,1603331002965399,1603331002965749,130
4,AA,0,0,NULL,NULL,NULL,NULL,6,6,1,1603331004321147,1603331004321497,313
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,AA,0,0,NULL,NULL,NULL,NULL,38,38,1,1603338056645464,1603338056645814,1124088
11495,AA,0,0,NULL,NULL,NULL,NULL,38,38,1,1603338057867388,1603338057867738,1124292
11496,AA,0,0,NULL,NULL,NULL,NULL,38,38,1,1603338057872257,1603338057872607,1124295
11497,AA,0,0,NULL,NULL,NULL,NULL,38,38,1,1603338059213551,1603338059213901,1124490


In [6]:
bin_interval_ms = 500
summarize_historical_trades_df(symbol_historical_trades_df, bin_interval_ms)

,MinPrice,MaxPrice,OpenPrice,ClosePrice,Volume
time,,,,,
2020-10-22 01:43:21.500,6.0,6.0,6.0,6.0,2.0
2020-10-22 01:43:22.000,6.0,6.0,6.0,6.0,0.0
2020-10-22 01:43:22.500,6.0,6.0,6.0,6.0,2.0
2020-10-22 01:43:23.000,6.0,6.0,6.0,6.0,0.0
2020-10-22 01:43:23.500,6.0,6.0,6.0,6.0,0.0
...,...,...,...,...,...
2020-10-22 03:40:57.000,38.0,38.0,38.0,38.0,0.0
2020-10-22 03:40:57.500,38.0,38.0,38.0,38.0,2.0
2020-10-22 03:40:58.000,38.0,38.0,38.0,38.0,0.0


## 1.3 Backtest Mean Reversion Trader

We can backtest the choice of parameters (`moving_average` and `threshold`) that we have selected by evaluating performance on historical data. 

`backtest()` Arguments:
- symbol_historical_trades_df `pd.DataFrame` - Stock time series dataframe to backtest on. 
- num_shares `int`: Number of shares to buy or sell. 
- init_capital `int`: Amount of capital to start with. 
- init_shares `int`: Number of shares to start with. 
- Specific to mean reversion algorithm:
 - ma `int` - Number of steps in the moving average window 
 - threshold `float` - % above or below the moving average at which we will place orders 

`backtest()` returns:
- ROI (%)
- a list of (time index, action, price) tuples

In [7]:
# Initial capital and shares we will use for the backtest. 
init_capital = 100000
init_shares = 1000

# Set up trading algorithm parameters.
num_shares = 10
moving_average = 29
threshold = 1

# Set up mean reversion trader (Note: we set the trader parameter to None since we are offline.)
trader = None
mrt = meanReversionTrader(trader, [symbol])

# Run the backtest.
roi, action_list = mrt.backtest(symbol_historical_trades_df,
             num_shares, 
             init_capital, 
             init_shares, 
             ma=moving_average, 
             threshold=threshold)

print(action_list)
print("Mean Reversion Trader, ma={ma} threshold={threshold} ----- ROI={roi}%".format(
    ma=moving_average, threshold=threshold, roi=roi))

Running in offline mode. Could not set active symbols.
[(29, 'Sell', 38.0), (30, 'Sell', 38.0), (31, 'Sell', 38.0), (32, 'Sell', 38.0), (33, 'Sell', 38.0), (34, 'Sell', 38.0), (35, 'Sell', 38.0), (36, 'Sell', 38.0), (37, 'Sell', 38.0), (38, 'Sell', 38.0), (39, 'Sell', 38.0), (40, 'Sell', 38.0), (41, 'Sell', 38.0), (42, 'Sell', 38.0), (43, 'Sell', 38.0), (44, 'Sell', 38.0), (45, 'Sell', 38.0), (46, 'Sell', 38.0), (47, 'Sell', 38.0), (88, 'Sell', 56.0), (89, 'Sell', 56.0), (90, 'Sell', 56.0), (91, 'Sell', 56.0), (92, 'Sell', 56.0), (93, 'Sell', 56.0), (94, 'Sell', 56.0), (95, 'Sell', 56.0), (96, 'Sell', 56.0), (97, 'Sell', 56.0), (98, 'Sell', 56.0), (99, 'Sell', 55.0), (100, 'Sell', 55.0), (101, 'Sell', 56.0), (102, 'Sell', 56.0), (103, 'Sell', 56.0), (104, 'Sell', 56.0), (105, 'Buy', 38.0), (106, 'Buy', 38.0), (107, 'Buy', 38.0), (108, 'Buy', 38.0), (109, 'Buy', 38.0), (110, 'Buy', 38.0), (111, 'Buy', 38.0), (112, 'Buy', 38.0), (113, 'Buy', 38.0), (114, 'Buy', 38.0), (115, 'Buy', 38.0),

# 2. Connect to Live Exchange

We will set up a connection to CloudEx which can be established at any time during trading hours. Let's see how we can connect and query for our historical order information.

## 2.1 Create Trader Object

This object will instantiate a connection to a gateway in CloudEx. This Gateway will relay order to the matching engine and market data to your client.

**Arguments to `Trader` constructor:**
- gateway_ip `str` - The IP address for the gateway assigned to you
- client_id `str` - Your client identifier
- client_token `str` - Your client token

In [8]:
# Get relevant fields from VM-specific config. Token is yours only, so don't make it public.
gateway_ip = config["gateway_ip"]
client_id = config["client_id"]
client_token = config["client_token"]

# Clear any existing data locally.
redis_api = redis.Redis()
redis_api.flushall();

# Create trader object.
trader = cloud_ex.Trader(gateway_ip, client_id, client_token)

## 2.2 Get My Outstanding and Historical Orders

In [9]:
# Get outstanding orders.
outstanding_orders = cloud_ex.MapStringOrder()
trader.GetOutstandingOrders(outstanding_orders)

print("You have {} outstanding orders.".format(len(outstanding_orders)))

# Transform outstanding orders into a DataFrame
outstanding_orders = OutstandingOrderDF(outstanding_orders)
outstanding_orders


# Get my historical orders (max of 100,000 historical orders).
my_historical_orders = cloud_ex.VectorOrder()

trader.GetAllHistoricalOrders(my_historical_orders)
print("You have submitted a total of {} order(s).".format(len(my_historical_orders)))
my_historical_orders_df = OrderDF(my_historical_orders)

my_historical_orders_df

You have 0 outstanding orders.
You have submitted a total of 0 order(s).


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares


## 2.3 Get My Historical Trades

In [10]:
# Get all my historical trades (max of 100,000 historical trades)
my_historical_trades = cloud_ex.VectorTrade()

trader.GetAllHistoricalTrades(my_historical_trades)
print("You have made a total of {} trade(s).".format(len(my_historical_trades)))
my_historical_trades_df = TradeDF(my_historical_trades)

my_historical_trades_df

You have made a total of 0 trade(s).


,Symbol,BuyerSerialNum,SellerSerialNum,BuyerOrderID,SellerOrderID,BuyerClientID,SellerClientID,ExecPrice,CashTraded,SharesTraded,CreationTimestamp,ReleaseTimestamp,TradeSerialNum


## 2.4 Get My Portfolio Matrix

In [11]:
portfolio_mat = cloud_ex.MapStringInt()

trader.GetPortfolioMatrix(portfolio_mat)

print(portfolio_mat)

MapStringInt{AA: 10000, AB: 10000, AC: 10000, AD: 10000, AE: 10000, AF: 10000, AG: 10000, AH: 10000, AI: 10000, AJ: 10000, AK: 10000, AL: 10000, AM: 10000, AN: 10000, AO: 10000, AP: 10000, AQ: 10000, AR: 10000, AS: 10000, AT: 10000, AU: 10000, AV: 10000, AW: 10000, AX: 10000, AY: 10000, AZ: 10000, BA: 10000, BB: 10000, BC: 10000, BD: 10000, BE: 10000, BF: 10000, BG: 10000, BH: 10000, BI: 10000, BJ: 10000, BK: 10000, BL: 10000, BM: 10000, BN: 10000, BO: 10000, BP: 10000, BQ: 10000, BR: 10000, BS: 10000, BT: 10000, BU: 10000, BV: 10000, BW: 10000, BX: 10000, BY: 10000, BZ: 10000, CA: 10000, CASH: 50000000, CB: 10000, CC: 10000, CD: 10000, CE: 10000, CF: 10000, CG: 10000, CH: 10000, CI: 10000, CJ: 10000, CK: 10000, CL: 10000, CM: 10000, CN: 10000, CO: 10000, CP: 10000, CQ: 10000, CR: 10000, CS: 10000, CT: 10000, CU: 10000, CV: 10000, CW: 10000, CX: 10000, CY: 10000, CZ: 10000, DA: 10000, DB: 10000, DC: 10000, DD: 10000, DE: 10000, DF: 10000, DG: 10000, DH: 10000, DI: 10000, DJ: 10000, DK:

# 3. Run Trading Algorithm in Live Environment

Now that we have an algorithmic trader that we evaluated on historical data, let's see how we can place trades on CloudEx using the mean reversion trader we set up in the section 2.

## 3.1 Create and Start Mean Reversion Trader

First, we will create a mean reversion trader.
Once created the trader will pull data from CloudEx, process the data
and lastly place buy or sell orders.

The `meanReversionTrader` object is created with the following parameters:
- trader `cloud_ex.Trader`: The CloudEx base trader object 
- symbol_list `List(str)`: A list of symbols whose market data we would like to subscribe to

We then invoke the trading loop with the following arguments
- symbol `str` - The symbol we would like to trade
- num_shares `int` - Number of shares to buy/sell per order
- num_orders `int` - Number of total orders to place 
- wait_interval `float` - Time to wait between placing orders (s) 
- ma `int` - Number of steps in the moving average window 
- threshold `float` - % above or below the moving average at which we will place orders 

When you have the above information, you can run the order loop using the `meanReversionTrader.trade` function.

In [ ]:
class meanReversionTrader(AlgorithmicTrader):
    def __init__(self, trader, symbol_list, bin_interval_ms=500):
        """
        Mean Reversion Trader Class.

        :param trader(cloud_ex.Trader): CloudEx's base Trader object.
        :param symbol_list: List of ticker symbols (str) to fetch data for.
        :param bin_interval_ms: Frequency with which to bin trading data in milliseconds (int).
        """
        # Initialize AlgorithmicTrader.
        AlgorithmicTrader.__init__(self, trader, symbol_list, bin_interval_ms=bin_interval_ms)

    def algorithm(self, df, **kwargs):
        """ 
        Calculate buy sell points for an equity.
        A buy is triggered when the moving average is threshold% below the price.
        A sell is triggered when the moving average is threshold% above the price.
        For this implementation we evaluate the stock price at the close of each bin_interval_ms time interval.

        :param df: Dataframes with prices for an equity (pd.DataFrame).
        Mean reversion specific arguments:
          :param ma: Number of bins to consider in the moving average (int).
          :param threshold: Amount ma is below/above price (as a percentage) for which to buy/sell (float).

        Returns: 
            An (action, price) pair.
            
            'action' is None when no action is determined to be taken. This happens either when we do not have enough
            data to build a moving average estimate or if the price is within a small threshold above/below
            the moving average. In this case, 'price' is also None.

            'action' is 'Buy' when the price rises above the threshold defined for the moving average. 
                The buy price is set to the most recent closing price.
                
            'action' is 'Sell' when the price drops below the threshold defined for the moving average. 
                The sell price is set to the most recent closing price.
        """
        # Unpack keyword arguments.
        ma = kwargs['ma']
        threshold = kwargs['threshold']
        
        # Handle case when data not available to build moving average.
        if len(df) < ma:
            return None, None
        
        # Select the last ma rows from our dataframe for processing.
        df = df.iloc[-1*ma:]

        df['MA'] = df['ClosePrice'].rolling(ma).mean()
        row = df.iloc[-1]

        # Check if moving average price is defined.
        # This value will be undefined for the first `ma` steps.
        if np.isnan(row['MA']):
            return None, None

        buy_cutoff = (1.0 - threshold / 100.0) * row['MA']
        sell_cutoff = (1.0 + threshold / 100.0) * row['MA']
        p_s = row['ClosePrice']
        if p_s <= buy_cutoff:
            return 'Buy', p_s
        elif p_s >= sell_cutoff:
            return 'Sell', p_s
        else:
            return None, None

In [13]:
# Algorithm Settings.
symbol = 'AA'
num_shares = 100
num_orders = 300
wait_interval = 0.5
moving_average = 29
threshold = 1

# Create and run mean reversion trader.
mrt = meanReversionTrader(trader, [symbol])   #the default parameter bin_interval_ms=500
mrt_order_id_list = mrt.trade(symbol,
                      num_shares,
                      num_orders,
                      wait_interval,
                      ma=moving_average,
                      threshold=threshold)
num_submitted_orders = len([order_id for order_id in mrt_order_id_list if order_id])
print("\n\nSubmitted orders in {} out of {} iterations".format(num_submitted_orders, len(mrt_order_id_list)))

no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
2 new trades
Did not submit an order in this iteration.
2 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
1 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
3 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
2 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
2 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order

Placing trade, symbol= AA  price= 38.0  submit_num_shares= 100  buy= False


1 new trades


Placing trade, symbol= AA  price= 38.0  submit_num_shares= 100  buy= False


1 new trades


...

no new trades


Placing trade, symbol= AA  price= 6.0  submit_num_shares= 100  buy= True


2 new trades


Placing trade, symbol= AA  price= 6.0  submit_num_shares= 100  buy= True




Submitted orders in 135 out of 300 iterations
